In [4]:
import os
import pandas as pd
from arango import ArangoClient

In [5]:
os.getcwd()

'/home/jovyan/centrality'

In [6]:
raw_df = pd.read_csv('../data/raw.csv')

In [7]:
client = ArangoClient(hosts="https://2ae4f052d710.arangodb.cloud:8529")

db = client.db('machine_learning', username='lab_test', password='z-rRdN-Enf4qQwybGiVdbG')

In [8]:
# db.delete_view('degree_centrality_view')

In [24]:
# if not db.has_view('degree_centrality_view'):
# if 'degree_centrality_view' not in [view['name'] for view in db.views()]:
#     degree_centrality = db.create_view(
#         name='degree_centrality_view', 
#         view_type='arangosearch', 
#         properties={
#             'links': {
#                 'organizations': {
#                     'includeAllFields': True
#                 }
#             },
#             'consolidation_interval_msec': 1000  # Refresh the view every second
#         }
#     )
# else:
#     degree_centrality = db.update_view(
#         name='degree_centrality_view', 
#         properties={
#             'links': {
#                 'organizations': {
#                     'includeAllFields': True
#                 }
#             },
#             'consolidation_interval_msec': 1000  # Refresh the view every second
#         }
#     )
# degree_centrality = db.view('degree_centrality_view')

# degree_centrality.update_properties({
#     'links': {
#         'organizations': {
#             'includeAllFields': True
#         }
#     }
# })

aql_query = """
FOR v IN organizations
    LET outbound_degree_centrality = LENGTH(
        FOR e IN transactions
            FILTER e._from == v._id
            RETURN e
    )
    LET inbound_degree_centrality = LENGTH(
        FOR e IN transactions
            FILTER e._to == v._id
            RETURN e
    )
    LET total_degree_centrality = outbound_degree_centrality + inbound_degree_centrality

    RETURN {
        _key: v._key,
        inbound_degree: inbound_degree_centrality,
        outbound_degree: outbound_degree_centrality,
        total_degree: total_degree_centrality
    }
"""

In [25]:
cursor = db.aql.execute(aql_query)
scores = list(cursor)

In [26]:
for score in scores:
    if score['_key'] == '0d47cd9e69a49f9664f8b8833e61f0d3':
        print(score)

{'_key': '0d47cd9e69a49f9664f8b8833e61f0d3', 'inbound_degree': 834, 'outbound_degree': 73, 'total_degree': 907}


In [27]:
(raw_df['sender_id'] == '0d47cd9e69a49f9664f8b8833e61f0d3').sum()

np.int64(73)

In [28]:
(raw_df['receiver_id'] == '0d47cd9e69a49f9664f8b8833e61f0d3').sum()

np.int64(834)

In [29]:
if db.has_collection('organization_organization_transaction_degree'):
    collection = db.collection('organization_organization_transaction_degree')
else:
    collection = db.create_collection('organization_organization_transaction_degree')

In [30]:
collection.insert_many(scores, overwrite_mode="replace", silent=True)

True